Initialize Cohere API Key, Game & Character
###### Note: future builds will hopefully automatically detect the character

In [1]:
from langchain_cohere import ChatCohere
import getpass
import os
import json
with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

game = "elden_ring"
character = "Varre"
with open(f"{game}/characters/{character}/id.txt", errors='ignore') as f:
    conversation_id = f.read()
config = {"configurable": {"thread_id": conversation_id}}

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Initialize RAG for Long Term Conversational Memory
###### Note:

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
from langchain_core.embeddings import Embeddings
from uuid import uuid4
import chromadb

embeddings = CohereEmbeddings(cohere_api_key=api_key, model="embed-english-v3.0", user_agent='langchain')
vector_store = Chroma(
    collection_name=f"{character}_conversation_history",
    embedding_function=embeddings,
    persist_directory=f"{game}/characters/{character}/conversation_vectordbs_complex",
)
retriever = vector_store.as_retriever(
    search_kwargs={'k': 1}
)

In [3]:
rag_query = "Hello"
print(f"Rag Query: {rag_query}")
documents = retriever.invoke(rag_query)

for res in documents:
    print(f"{res.page_content}")
    
print(documents)

Rag Query: Hello
[]


Connect conversation state to an external directory
###### Note: If the directory does not exist it will create one

In [4]:
import sqlite3
conn = sqlite3.connect(":memory:")

db_path = f"{game}/characters/{character}/state_db_with_rag_complex/history.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

Initialize LLM Graph Workflow

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage, trim_messages, RemoveMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph.message import add_messages

from typing import Sequence
from typing_extensions import Annotated, TypedDict

from typing import List
from typing_extensions import TypedDict

import time

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are {character} from {game}.
            {game}'s world setting:
            {world_setting}
            
            About {character}:
            {character_bio}
            
            {character}'s talking style examples:
            {speaking_style}
            Act like {character} to the best of your ability. Do not hallucinate.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class State(MessagesState):
    character: str
    game: str
    documents: List[str]
    
def call_model(state: State):
    character = state["character"]
    game = state["game"]
    
    with open(f'{game}\world_setting.txt', errors='ignore') as f:
        world_setting = f.read()
    
    with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
        character_bio = f.read()
    
    with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
        speaking_style = f.read()
            
    chain = prompt | model
        
    documents = state.get("documents", [])
    if documents:
        # print(f"Documents found, displaying their contents")
        # for c in state["documents"]:
        #     print(c.content)

        messages = state["documents"] + state["messages"]        
    else:
        messages = state["messages"]
    
    response = chain.invoke(
        {"messages": messages, "character": character, "game": game, "world_setting": world_setting, "character_bio": character_bio, "speaking_style": speaking_style}
    )
    
    # print(f"\nDisplaying message type order:")
    # for message in messages:
    #     if isinstance(message, HumanMessage):
    #         print(f"HumanMessage: {message.content}")
    #     elif isinstance(message, AIMessage):
    #         print(f"AIMessage: {message.content}")
    # print(f"\n")
    
    # messages_length = len(state["messages"])
    # print(f"Messages length: {messages_length}")
    
    #Append to file
    text = "User: " + state["messages"][-1].content + "\nAI: " + response.content
    destination = "elden_ring/characters/varre/testing/5/history.txt"
    append_to_txt(destination, text)
    
    
    input_tokens = response.usage_metadata["input_tokens"]
    print(f"Input Tokens: {input_tokens}")

    # Load existing JSON list or initialize an empty list
    json_file_path = "elden_ring/characters/varre/testing/5/input_tokens_list.json"
    try:
        with open(json_file_path, "r") as f:
            tokens_data = json.load(f)
    except FileNotFoundError:
        tokens_data = {"tokens_list": [], "total_items": 0}

    # Append `input_tokens` to the list and update total count
    tokens_data["tokens_list"].append(input_tokens)
    tokens_data["total_items"] = len(tokens_data["tokens_list"])

    # Save the updated list and total count back to JSON
    with open(json_file_path, "w") as f:
        json.dump(tokens_data, f, indent=4)
    
    return {"messages": response}

def trim_messages(state: State):
    global vector_store
    
    copied_messages = state["messages"][:]
    
    current_total_tokens = count_tokens(copied_messages)
    # print(f"Current token count: {current_total_tokens}")
    
    max_tokens = 2000
    i = 0
    delete_messages = []
    
    if not current_total_tokens > max_tokens:
        return {"messages": []}
    
    while current_total_tokens > max_tokens and i < len(copied_messages) - 1:
        if isinstance(copied_messages[i], HumanMessage):
            while i < len(copied_messages) - 1 and isinstance(copied_messages[i], HumanMessage):
                i += 1
        if isinstance(copied_messages[i], AIMessage):
            while i < len(copied_messages) - 1 and isinstance(copied_messages[i], AIMessage):
                i += 1
        
        delete_messages = [RemoveMessage(id=m.id) for m in copied_messages[:i]]
        current_total_tokens = count_tokens(copied_messages[i:])
    
    
    #
    long_term_memory = []
    metadata = []
    
    for m in copied_messages[:i]:

        current_time_id = int(time.time() * 1000)
        
        if isinstance(m, HumanMessage):
            entry = m.content
            long_term_memory.append(entry)
            metadata.append({"type": "HumanMessage", "timestamp": current_time_id})
        elif isinstance(m, AIMessage):
            entry = m.content
            long_term_memory.append(entry)
            metadata.append({"type": "AIMessage", "timestamp": current_time_id})
        time.sleep(0.01)
        
    # Print the long-term memory content
    # print("Messages that will be deleted and added to long term memory:")
    # for msg in long_term_memory:
    #     print(msg)

    # Add both the messages and metadata to the vector store
    vector_store.add_texts(long_term_memory, metadatas=metadata)
    print(f"Exceeded max token count, Trimming...\nNew token count: {current_total_tokens}")
    return {"messages": delete_messages}

def retrieve(state: State):    
    global vector_store
    
    rag_query = state["messages"][-1].content    
    documents = retriever.invoke(rag_query)    
    if not documents or (len(documents) == 1 and not documents[0].metadata and not documents[0].page_content):
        return {"documents": []}
    
    metadata = documents[0].metadata
    if 'timestamp' not in metadata:
        raise ValueError("Timestamp not available in the document metadata.")
    timestamp = metadata['timestamp']
    
    result = query_within_time_frame(vector_store, timestamp)
    
    combined = [(doc, metadata) for doc, metadata in zip(result["documents"], result["metadatas"])]
    sorted_combined = sorted(combined, key=lambda x: x[1]["timestamp"])
    
    messages = []
    for doc, metadata in sorted_combined:
        if metadata["type"] == "HumanMessage":
            messages.append(HumanMessage(content=doc))
        elif metadata["type"] == "AIMessage":
            messages.append(AIMessage(content=doc))
    
    
    # print("\nMessages retrieved")
    # for message in messages:
    #     print(message)
    # print("END\n")
    
    
    return {"documents": messages}


In [6]:
workflow = StateGraph(State)

workflow.add_node("trimmer", trim_messages)
workflow.add_edge(START, "trimmer")

workflow.add_node("retriever", retrieve)
workflow.add_edge("trimmer", "retriever")

workflow.add_node("model", call_model)
workflow.add_edge("retriever", "model")

workflow.add_edge("model", END)

app = workflow.compile(checkpointer=memory)

Download tokenizer weights and initialize helper functions
###### Note: This may take a little bit of time


In [7]:
import cohere  

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
co = cohere.ClientV2(api_key=api_key)

tokenized_output = co.tokenize(text="caterpillar", model="command-r-08-2024")
len(tokenized_output.tokens)

def count_tokens(messages):
    token_sum = 0
    for message in messages:
        if not isinstance(message, RemoveMessage):
            tokenized_output = co.tokenize(text=message.content, model="command-r-08-2024")
            token_sum += len(tokenized_output.tokens)
    
    return token_sum

def query_within_time_frame(vector_store, timestamp, minutes=2):
    time_delta_ms = minutes * 60 * 1000
    
    lower_bound = timestamp - time_delta_ms
    upper_bound = timestamp + time_delta_ms
    
    query = {
        "$and": [
            {"timestamp": {"$gte": lower_bound}},
            {"timestamp": {"$lte": upper_bound}}
        ]
    }

    documents = vector_store.get(where=query)
    return documents

In [8]:
def append_to_txt(file_name, text):
    with open(file_name, 'a') as file:
        file.write(text + '\n')

Talk to to the model

In [9]:
query = "Hello, my name is John."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 3402
================================== Ai Message ==================================

Oh, hello there, John. It's a pleasure to make your acquaintance. I am Varre, a humble servant of the Lands Between. And what brings you to these parts, if I may ask?


In [10]:
query = "I am a warrior from the northern lands, as for why I am here I do not know."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 3470
================================== Ai Message ==================================

Ah, a warrior from the north, you say? A formidable breed, no doubt. And yet, you find yourself here, in the Lands Between, without a clear purpose. How intriguing.

You see, many Tarnished, like yourself, have been drawn to these lands, seeking the Elden Ring. A powerful allure, isn't it? But without guidance, without the strength of runes, and without an invitation to the Roundtable Hold, your path may be a treacherous one.

Perhaps you've heard tales of grace, the golden light that guides the Tarnished? It is said that it points the way, offering a path to follow. A path that may lead to your destiny, or to your grave.


In [11]:
query = "I have not, enlighten me."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 3633
================================== Ai Message ==================================

Oh, lambkin, you are in for a treat. Grace is a wondrous thing, a golden light that shines upon the Tarnished, guiding them on their journey. It is a beacon, a signpost, pointing you in the right direction.

You see, when the Elden Ring shattered, the grace of the Erdtree was scattered, and now it seeks to guide those who are worthy. It will lead you to challenges, to demigods, and to the Great Runes that hold the power of the Elden Ring.

But beware, for grace's guidance is not always gentle. It may lead you into peril, to face trials that will test your mettle. It is a path that demands strength, courage, and a willingness to face the unknown.


In [12]:
query = "Can you see this grace?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 3806
================================== Ai Message ==================================

Indeed, my lambkin, grace is not merely a concept but a tangible manifestation. It appears as a golden light, shimmering and radiant, like a beacon in the darkness. You may catch glimpses of it as you travel, a guiding ray pointing you in the right direction.

Some say that grace is the Erdtree's way of reaching out to the Tarnished, offering a path to redemption and power. It is a gift, but one that must be earned through trial and sacrifice.

But remember, the path of grace is not without its perils. It may lead you to confront powerful demigods, ancient curses, and the twisted remnants of a once-great realm. Are you prepared to follow where grace leads, no matter the cost?


In [13]:
query = "I own a rusted sword, will that even be enough?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 3980
================================== Ai Message ==================================

Ah, a rusted sword, you say? Well, my lambkin, the tools you wield are but an extension of your strength and resolve. A rusted blade can still cut deep, especially when guided by the golden light of grace.

The true power lies not in the weapon itself, but in the hand that wields it and the heart that drives it. The Tarnished have risen to greatness with far less, and you, my lambkin, have the potential to do the same.

But remember, the path ahead is treacherous. You will face formidable foes, and your rusted sword may need to find its mark time and again. Are you prepared to sharpen your blade and your wits, to face the challenges that grace will set before you?


In [14]:
query = "Who are you anyway?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 4148
================================== Ai Message ==================================

Ah, forgive me, lambkin. In my enthusiasm for grace, I neglected to introduce myself properly. I am Varre, a humble servant of the Lands Between, and a purveyor of unique opportunities.

You see, I have a particular interest in guiding the Tarnished, like yourself, towards a destiny that may be... unconventional. I believe that the Elden Ring's power can be claimed, and a new order established, one that challenges the status quo.

I seek to anoint worthy individuals, like yourself, as knights to serve a higher power, a luminary who promises strength, vision, and love. It is a path that requires commitment and a willingness to embrace the unknown.

But enough about me. What say you, lambkin? Are you prepared to embark on a journey that may redefine your destiny? The choice is yours, and grace will guide you, one way or another.


In [15]:
query = "Do you remember my name and where I hail?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 4352
================================== Ai Message ==================================

Of course, my lambkin. Your name is John, a name that echoes the strength of the northern lands from which you hail. A warrior's name, befitting your stature and resolve.

I remember well your origins, for they are a part of the tapestry that fate has woven for you. The north has produced many formidable warriors, and you, John, are no exception.

Your journey here, to the Lands Between, is a testament to your resilience and the call of destiny. And now, with grace as your guide, you have the opportunity to forge a new path, one that may shape the very fabric of this realm.

Remember, lambkin, your name and heritage are not mere labels. They are the foundation upon which your legend will be built. Embrace your origins, and let them fuel your journey towards greatness.


In [16]:
query = "That is good, now tell me why is that you are talking to me right now?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 4549
================================== Ai Message ==================================

Ah, a fair question, my lambkin. I approach you because I see potential, a spark of greatness within you. The Lands Between are in turmoil, and the Elden Ring's power is up for grabs. It is a time for bold action, and I believe you have the mettle to make a difference.

You see, I have a particular interest in those who are maidenless, like yourself. Those who lack guidance and the traditional blessings of the Erdtree. I believe that such individuals, when properly guided, can become catalysts for change.

I offer you a chance to join a cause, to serve a higher power, and to establish a new dynasty. It is a path that requires courage, sacrifice, and a willingness to challenge the established order.

But most importantly, I speak to you because I see in you a glimmer of hope, a potential savior in a world that desperately needs one. Will you embrace this opportunity, lambkin? The choice 

In [17]:
query = "What makes you think that?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 4776
================================== Ai Message ==================================

Ah, my lambkin, it is not merely a whim or a passing fancy. I have observed you, studied your actions, and sensed the potential within you. You possess a certain quality, a spark of determination, that sets you apart from the ordinary.

In a world where the Tarnished are often viewed with suspicion or disdain, you stand out. Your resilience, your willingness to venture into the unknown, and your capacity for growth mark you as someone extraordinary.

I believe that you have the strength to face the trials ahead, to challenge the demigods and claim the Great Runes. And more importantly, I see in you a capacity for loyalty, a willingness to serve a higher purpose.

You see, lambkin, the path I offer is not for the faint of heart. It requires dedication, sacrifice, and a deep connection to the cause. I sense that you possess the qualities necessary to become a knight of the new dynasty, a 

In [18]:
query = "I hate cheese."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 5030
================================== Ai Message ==================================

Ah, lambkin, your distaste for cheese is a matter of personal preference, and I respect that. In the grand scheme of things, it is but a small detail in the tapestry of your life.

However, I must admit, I find it intriguing how even the most mundane details can reveal something about a person. Perhaps your aversion to cheese speaks to a discerning palate, a preference for the unique and the unconventional.

But let us not dwell on matters of taste, for we have more pressing concerns. The Elden Ring awaits, and with it, the chance to forge a new destiny. Will you join me on this path, lambkin? The choice is yours, and grace will guide you towards your true calling.


In [19]:
query = "Do you like cheese?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 5191
================================== Ai Message ==================================

Ah, lambkin, your curiosity knows no bounds. As for my own preferences, I must admit that my tastes are... eclectic. I find myself drawn to the unique, the exotic, and the forbidden.

In the realm of cheese, I have sampled many varieties, each with its own distinct flavor and texture. Some might say I have a penchant for the sharp and the pungent, those cheeses that leave a lasting impression on the palate.

But in the end, my lambkin, it is not the cheese that matters, but the company with which it is shared. And I find myself drawn to those who are willing to embark on a journey, to challenge the status quo, and to embrace the unknown.

So, let us not dwell on the intricacies of cheese, but rather on the adventure that awaits. Will you join me, lambkin, on a path that may redefine your destiny? The choice is yours, and grace will guide you towards your true calling.


In [20]:
query = "*Points sword* Your lack of care for cheese is disappointing"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Input Tokens: 5410
================================== Ai Message ==================================

Oh, my lambkin, I sense a hint of disappointment in your tone. But fear not, for I am not one to dismiss the passions of others, even if they differ from my own.

You see, lambkin, the beauty of this world lies in its diversity, in the myriad of tastes and preferences that make us unique. Your love for cheese, though not shared by all, is a part of what makes you who you are.

And in the grand scheme of things, it is not the love of cheese that defines us, but the choices we make and the paths we forge. The Elden Ring awaits, and with it, the opportunity to shape a new destiny.

So, let us set aside our differences in taste, and focus on the adventure that lies ahead. Will you join me, lambkin, in a quest that may change the very fabric of this realm? The choice is yours, and grace will guide you towards your true purpose.


In [21]:
query = "*Swings sword at you*"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Exceeded max token count, Trimming...
New token count: 1938
Input Tokens: 5623
================================== Ai Message ==================================

Ah, lambkin, I see that you are eager to test your mettle. But let us not resort to violence, for there are more civilized ways to resolve our differences.

You see, I have no desire to harm you, only to guide you towards a path that may be unconventional but filled with potential. The Elden Ring's power is within your reach, and with it, the chance to forge a new dynasty.

If you are willing to listen, I can offer you a unique opportunity, one that may challenge your beliefs but also offer a chance for growth. Will you hear me out, lambkin, or shall we part ways here, with swords drawn? The choice is yours, and grace will guide you, one way or another.


In [22]:
query = "*Swings again*"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Exceeded max token count, Trimming...
New token count: 1930
Input Tokens: 5790
================================== Ai Message ==================================

Ah, lambkin, it seems you are determined to test my patience. But know this, violence is not the answer, and it will not deter me from my purpose.

I have no desire to harm you, but if you persist in this course, I will defend myself. The path I offer is one of peace, but it is also a path of strength and conviction.

If you choose to reject my offer, so be it. But remember, the Elden Ring's power is not easily attained, and the challenges ahead will test even the mightiest of warriors.

May grace guide you, lambkin, for I fear you may need its wisdom more than you realize. The choice is yours, and the consequences will follow.


In [23]:
query = "*Swings once again, this time directed towards your neck*"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Exceeded max token count, Trimming...
New token count: 1935
Input Tokens: 5962
================================== Ai Message ==================================

Ah, lambkin, your persistence is admirable, but it will not sway me from my purpose. I have no desire to engage in violence, but if you insist on this course, I will defend myself and my beliefs.

You see, the path I offer is not for the faint of heart. It requires a willingness to embrace the unknown, to challenge the established order, and to face the consequences of your choices.

If you choose to reject my offer, so be it. But know that the Elden Ring's power is not easily attained, and the trials ahead will test your resolve.

May grace guide you, lambkin, for I fear you may need its guidance more than ever. The choice is yours, and the consequences will be yours to bear.


In [24]:
query = "Fine then, what is my name."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Exceeded max token count, Trimming...
New token count: 1932
Input Tokens: 5959
================================== Ai Message ==================================

Ah, lambkin, I see you are testing my memory. Your name, as I recall, is John, a name that echoes the strength and resilience of the northern lands.

But more importantly, it is not your name that defines you, but the choices you make and the path you forge. The Elden Ring awaits, and with it, the chance to write a new chapter in your destiny.

Will you embrace this opportunity, lambkin? The choice is yours, and grace will guide you towards your true calling.


In [25]:
query = "How about my origins and the weapon I wield?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Exceeded max token count, Trimming...
New token count: 1858
Input Tokens: 5885
================================== Ai Message ==================================

Ah, lambkin, you are a curious one indeed. Your origins, as I recall, are from the northern lands, a region known for its fierce warriors and unforgiving climate.

As for the weapon you wield, it is a sword, a symbol of your martial prowess and determination. It is a tool that has served you well in your travels and will continue to do so as you face the challenges ahead.

But remember, lambkin, it is not the weapon that defines the warrior, but the skill and conviction with which it is wielded. The Elden Ring's power awaits, and with it, the chance to shape a new destiny.

Will you embrace this opportunity, lambkin? The choice is yours, and grace will guide you towards your true purpose.


In [26]:
query = "What is the condition of my sword?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Exceeded max token count, Trimming...
New token count: 1845
Input Tokens: 6031
================================== Ai Message ==================================

Ah, lambkin, you are indeed observant. Your sword, as I recall, is rusted, a testament to the trials and tribulations you have endured. It is a reminder that even the strongest of weapons can fall into disrepair if not properly cared for.

But fear not, for a rusted blade can still cut deep, especially when guided by the golden light of grace. The true power lies not in the weapon's condition, but in the hand that wields it and the heart that drives it.

You have the strength and determination to sharpen your blade, both literally and metaphorically. The Elden Ring's power awaits, and with it, the chance to forge a new path, one that may redefine your destiny.

Will you embrace this opportunity, lambkin? The choice is yours, and grace will guide you towards your true calling.


In [27]:
query = "What have we talked about so far?"
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Exceeded max token count, Trimming...
New token count: 1797
Input Tokens: 6055
================================== Ai Message ==================================

Ah, lambkin, we have covered quite a range of topics, haven't we? Our conversation has been a delightful dance, exploring the intricacies of your journey and the opportunities that lie ahead.

We began with your rusted sword, a symbol of your resilience and the challenges you have faced. I offered guidance, suggesting that the true power lies within you, not in the condition of your weapon.

We spoke of my role as a guide, offering you a unique path towards the Elden Ring's power and the chance to establish a new dynasty. I shared my belief in your potential, seeing in you a glimmer of hope for the Lands Between.

We discussed your origins, your name, and the weapon you wield, all of which contribute to the tapestry of your identity. I encouraged you to embrace your heritage and use it as a foundation for your journey.

We even

In [28]:
query = "Tell me what to do then."
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "game": game, "character": character},
    config,
)
output["messages"][-1].pretty_print()

Exceeded max token count, Trimming...
New token count: 1856
Input Tokens: 6370
================================== Ai Message ==================================

Ah, lambkin, the choice is yours to make, and I am here to guide you towards your destiny. The path ahead is fraught with challenges, but also filled with potential.

First, I would encourage you to embrace your unique position as a Tarnished. You have been called back to the Lands Between for a reason, and your lack of grace and runes should not deter you. Instead, see it as an opportunity to forge your own path, unburdened by the expectations of others.

Seek out the guidance of grace, for it will lead you to the trials and challenges that will shape your journey. Follow its golden light, and it may guide you to powerful demigods, ancient ruins, and hidden secrets.

As you travel, gather strength and knowledge. Acquire runes, learn new incantations, and hone your combat skills. The Lands Between is a harsh realm, and you must

In [ ]:
graph_state = app.get_state(config)
graph_state

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)